# Creation table Olivier_it

`IT_Equipment_v2` : Table IT_Equipment provenant d'un fichier csv

### Fichier olivier_it contenant seulement les positions 

drop TABLE if EXISTS olivier_it_only_index ; 
CREATE TABLE `olivier_it_only_index` (
  `modele` varchar(255) DEFAULT NULL,
  `Type` varchar(100) DEFAULT NULL,
  `fabricant` varchar(255) DEFAULT NULL,
  `nom` varchar(255) DEFAULT NULL,
  `SHORT_NAME` varchar(100) DEFAULT NULL,
  `SERIAL_NUMBER` varchar(100) DEFAULT NULL,
  `ASSET_ID` varchar(100) DEFAULT NULL,
  `ASSET_TAG` varchar(100) DEFAULT NULL,
  `Asset_Number` varchar(100) DEFAULT NULL,
  `Nom_de_la_salle` varchar(255) DEFAULT NULL,
  `reperage_id` varchar(255) DEFAULT NULL,
  `Taille_U` varchar(100) DEFAULT NULL,
  `End_Move_Target` varchar(100) DEFAULT NULL,
  `Status_hpe`varchar(255) DEFAULT NULL,
  `Salle_Dest` varchar(100) DEFAULT NULL,
  `Rack_U_Dest` varchar(100) DEFAULT NULL,
  `olivier_index` varchar(100) DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

In [ ]:
DROP TABLE IF EXISTS `IT_Equipment_v2` ;
CREATE TABLE IT_Equipment_v2
as
SELECT * FROM `IT_Equipment`

`assets_filtree_v2` : Table assets_new (CMDB) filtré aux deux localisations : FRVLB0C2 / FRTCRAVA

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_v2` ;
CREATE TABLE assets_filtree_v2
as
select *
from `assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA') ;

In [ ]:
ALTER TABLE assets_filtree_v2
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

In [ ]:
UPDATE assets_filtree_v2
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

In [ ]:
DROP TABLE IF EXISTS `IT_Equipment_olivier_no_match` ;
CREATE TABLE IT_Equipment_olivier_no_match
as
SELECT it.*
FROM `IT_Equipment_v2` it
LEFT JOIN `olivier_it_only_index` oit on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

where it.nom_de_la_salle in ('P1', 'P2', 'P3', 'P4') and oit.olivier_index is null
ORDER BY Reperage_ID asc

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced` ;
CREATE TABLE assets_filtree_reduced
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4', '0')

`IT_Equipment_match_position_reduced_v3` : groupement des lignes par positions et affichage d'un seul asset_id par positions + nombres d'equipements sur la même position

In [ ]:
ALTER VIEW IT_Equipment_match_position_reduced_v3
as
SELECT it.nom_de_la_salle, it.reperage_id, count(it.reperage_id) as RowNumber,
		concat(aa.asset_id) asset_id

FROM   `olivier_it_only_index` it  
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
GROUP BY it.reperage_id

In [ ]:
ALTER TABLE olivier_it_only_index
ADD COLUMN STATUS_2 varchar(255),
ADD COLUMN Nom_de_baie varchar(255),
ADD COLUMN Slot_u varchar(255),
ADD COLUMN STATUS_3 varchar(255),
ADD COLUMN ASSET_ID_OK_POSITION varchar(255),
ADD COLUMN status varchar(255) ;

### `Colonnes de status differentes`

In [ ]:
ALTER TABLE olivier_it_only_index 
ADD COLUMN status_asset_id varchar(255),
ADD COLUMN status_position varchar(255),
ADD COLUMN status_name varchar(255),
ADD COLUMN status_etat varchar(255),
ADD COLUMN status_global varchar(255),
ADD COLUMN status_etat_hpe varchar(255),
ADD COLUMN status_spec varchar(255),
ADD COLUMN status_cc varchar(255) ;

In [ ]:
UPDATE `olivier_it_only_index`
set
Nom_de_baie = SUBSTRING_INDEX(reperage_id,':',1) ,
Slot_u = SUBSTRING_INDEX(reperage_id,'Slot',-1) ;

# UPDATE

## `Status` 

attribut `Status` :

    - Commun : Equipment present sur fichier Olivier_it & It_equipment
    - Non trouvé : Equipment non present sur It_equipment

In [ ]:
UPDATE `olivier_it_only_index` oit
INNER JOIN `IT_Equipment_v2` it on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

SET oit.status = 'Commun'

where it.nom_de_la_salle in ('P1', 'P2', 'P3', 'P4')

In [ ]:
UPDATE `olivier_it_only_index` oit
LEFT JOIN `IT_Equipment_v2` it on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

set oit.status = 'Non trouvé'

where it.reperage_id is null

### `MATCH POSITION`

attribut `status_position` : 

    - "Bonne localisation"
    - "KO localisation" : impossibilité de trouver la position dans la CMDB (assets_filtrée)

In [ ]:

UPDATE `olivier_it_only_index` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id 
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position = 'Bonne localisation', 
it.status_spec = concat("nb of equipement : ",it2.RowNumber)

WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
UPDATE `olivier_it_only_index` it 
SET it.status_position = 'KO localisation'
where it.status_position is null

# `Status : Etat`

attribut `status_etat` : Si un équipment est repéré (par son asset_id et/ou sa position ) 

    - "Trouvé - bon asset id"
    - "Non Trouvé - Mauvais Asset ID"
    - "KO Etat" - Aucun ou plusieurs equipments sur la même position, impossible d'établir le lien

In [ ]:

UPDATE `olivier_it_only_index` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id 
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position = 'Bonne localisation'
WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

# `Status : Etat`

attribut `status_etat` : Si un équipment est repéré (par son asset_id et/ou sa position ) 

    - "Trouvé - bon asset id"
    - "Non Trouvé - Mauvais Asset ID"
    - "KO Etat" - Aucun ou plusieurs equipments sur la même position, impossible d'établir le lien

In [ ]:
UPDATE `olivier_it_only_index` it
LEFT JOIN `assets_filtree_reduced` aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  

SET it.status_etat = if(it.status_spec = "nb of equipement : 1",aa.etat, it.status_spec) 

